# day5

> Day 5: Supply Stacks


The elves want to know **which crate will end up on top of each stack**.

The input is a picture of the crates and a list of crane instructions for moving them. Example:
```    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
```

Stack 1 has two crates: [N] and [Z]. Stack 2 has 3 crates; stack 3 has 1. 

* Step 1: move 1 crate from stack 2 to stack 1.
* Step 2: move **3 crates** from 1 to 3

::: {.callout-note}
Crates are moved **one at a time**. 
:::

In [ ]:
#| default_exp day5

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

# Example:
Here is the starting position again.

In [ ]:
example = """
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
""".strip("\n")   # Drop leading blank
print(example)

    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2


Okay, this is really two inputs separated by a blank line (`\n\n`).

1. The first is the **start pos**, a **column-oriented** list with headers at the bottom.
2. The second is the **move list**, effectively a **CSV** of `#, from_col, to_col`.

## Starting Position

In [ ]:
start = example.split("\n")
print("\n".join(start))

    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2


The input data is in two parts split by `\n\n`:

* **state** is in rows 1-4, columns 1, 5, 9, ..., $N-1$.  **Count by 4**
* **plan** is in rows 6-9, basically CSV.

In [ ]:
# Confirm the blanks go to the end of row
start[0][9]

' '

In [ ]:
#| export
#| code-fold: true
def get_state(rowdata: list[str]  # Data as if readlines
            )-> list[list[str]]:  # Row-oriented list, top-down
    ncols = len(rowdata[0])
    nrows = rowdata.index("") - 1
    return [[row[i] 
             for i in range(1, ncols, 4)]
            for row in rowdata[:nrows]]

def print_state(state: list[list[str]])-> None:
    """Print a picture of the state. Stacks should be vertical."""
    print("\n".join(" ".join(x) for x in state))
    print(" ".join(f"{i+1}" for i in range(len(state[0]))))


In [ ]:
state = get_state(start)
print_state(state)

  D  
N C  
Z M P
1 2 3


But it became clear I'd want that as actual stacks.

In [ ]:
import numpy as np
from collections import deque

Convert picture to list of stacks, removing blanks. 
If we kept the blanks this would be a simple `.T` transpose in numpy,
but we want ragged stacks or we will stack crates on air. 

In [ ]:
#| exports
#| code-fold: true

def stackify(state: list[list] # Crate state in visual format
            )-> list[deque]:   # State as list of stacks, no blanks
    """Convert row-oriented input to compact stacks, top=right."""
    nrows, ncols = len(state), len(state[0])
    return [deque([state[row][col] for row in range(nrows-1,-1,-1)
                if state[row][col] != " "])
            for col in range(ncols)]

def print_stacks(stacks: list[deque])-> None:
    """Print a horizontally-oriented picture of stacks"""
    N = max(len(row) for row in stacks)
    pad = ["  "*(N - len(row)) for row in stacks]
    for i, stack in enumerate(stacks):
        print(f"{(i+1)} {' '.join(stack)}{pad[i]}")

In [ ]:
#| test
_ = [[' ', 'D', ' '],
     ['N', 'C', ' '],
     ['Z', 'M', 'P'],
     ['1', '2', '3']]

assert stackify(_) == [deque(['1', 'Z', 'N']), deque(['2', 'M', 'C', 'D']), deque(['3', 'P'])]

In [ ]:
stacks = stackify(state)
print_stacks(stacks)

1 Z N  
2 M C D
3 P    


## Move List

In [ ]:
def get_moves(rowdata: list[str]  # As from readlines
             )-> list[list[int]]: # [[n, from_col, to_col], ...
    """Extract move data from input"""
    start = rowdata.index("") + 1
    moves = ("\n".join(rowdata[start:])
             .replace(" ", "")
             .replace("move", "")
             .replace("from", ",")
             .replace("to", ",")
             .split("\n")#[:-1]
            )
    return [x.split(",") for x in moves]

def print_moves(moves: list[list[int]])-> None:
    """Verbosely print a terse movelist."""
    print("\n".join(f"Move {n} from {old} to {new}" for n, old, new in moves))

In [ ]:
#| test
moves = get_moves(start)
print_moves(moves)

Move 1 from 2 to 1
Move 3 from 1 to 3
Move 2 from 2 to 1
Move 1 from 1 to 2


What is it to do a move? Line 1 moves the top crate `D` from stack 2 to stack 1.  Iterated on these with some REPL. Quickly became clear I wanted `state` to be stacks so made a quick change to rotate the lists 90º and use `deque`.  (Really could just use lists.)

Note the `-1` to correct for 0-indexing. Would be nicer to include a hidden stack 0 so we don't need that. Later.

In [ ]:
#| exports
#| code-fold: true

def move(
    n:    int|str,     # Move this many
    from_col: int|str, # From this stack
    to_col:   int|str, # To this stack
    pos:  list,        # Position -> Will be CHANGED!
    )-> None:
    """Move `n` crates from `from_col` to `to_col`. Modifies in place."""
    n, from_col, to_col = int(n), int(from_col), int(to_col)
    for i in range(n):
        move1box(pos[from_col - 1], pos[to_col - 1])

def move1box(
    from_stack: deque, # From this stack
    to_stack:   deque, # To this stack
    )-> None:          # Modifies in place
    """Move 1 crates from `from_stack` to `to_stack`."""
    to_stack.append(from_stack.pop())


::: {.callout-note} Took about 45min to get to this point with the _first_ versions of `move` and `move1box`, then had to stop for work. Given how straightforward this is, I'm suprised how long it takes me - documenting, thinking, little iterations & tests. 
:::

In [ ]:
stacks = stackify(state)
move(2, 1, 2, stacks)
stacks

[deque([]), deque(['M', 'C', 'D', 'N', 'Z']), deque(['P'])]

## Solve Example

### After Step 1:
Should be

In [ ]:
step1 = """
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
 """

In [ ]:
stacks = stackify(state)
move(*moves[0], stacks)
print_stacks(stacks)

1 Z N D
2 M C  
3 P    


After Step 2:

In [ ]:
step2 = """
        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3
 """

In [ ]:
move(*moves[1], stacks)
print_stacks(stacks)

1         
2 M C    
3 P D N Z


After Step 3:

In [ ]:
step3 = """
        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3
 """

In [ ]:
move(*moves[2], stacks)
print_stacks(stacks)

1 C M    
2         
3 P D N Z


End, after Step 4:

In [ ]:
step4 = """
        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3
 """

In [ ]:
move(*moves[3], stacks)
print_stacks(stacks)

1 C      
2 M      
3 P D N Z


Crates on top are: **[C] [M] [Z]**. 

Answer would be `CMZ`.

In [ ]:
#| export
#| code-fold: true
def top_crates(stacks: list[deque]  # List of stacks
              )-> str:              # Crates on top
    """Find the crates on top of each stack"""
    return "".join(stack[-1] for stack in stacks)

Test that

In [ ]:
#| test

assert top_crates(stacks) == "CMZ"

# Part 1

## Get the data


In [ ]:
with open("data/day5_input.txt") as f:
    data = [x.strip() for x in f.readlines()]
data[:3]

['[B]                     [N]     [H]',
 '[V]         [P] [T]     [V]     [P]',
 '[W]     [C] [T] [S]     [H]     [N]']

## Run

In [ ]:
stacks = stackify(get_state(data))
for _ in get_moves(data):
    move(*_, stacks)
top_crates(stacks)


'PSNRGBTFT'

# Part 2

**Oops!** This crane can pick up all the boxen at once!  It doesn't have to go one at a time.

Write a `moveall`, rerun, and say what the top crates will be.

In [ ]:
d = deque([3,4,5,6])
e = deque([1])
for x in reversed([d.pop() for i in range(3)]):
    e.append(x)
e

deque([1, 4, 5, 6])

In [ ]:
#| exports
#| code-fold: true

def moveall(
    n:    int|str,     # Move this many
    from_col: int|str, # From this stack
    to_col:   int|str, # To this stack
    pos:  list,        # Position -> Will be CHANGED!
    )-> None:
    """Move `n` crates from `from_col` to `to_col` AS A GROUP."""
    n, from_col, to_col = int(n), int(from_col), int(to_col)
    old, new = pos[from_col - 1], pos[to_col - 1]
    for x in reversed([old.pop() for i in range(n)]):
        new.append(x)

## Run

In [ ]:
stacks = stackify(get_state(data))
for _ in get_moves(data):
    moveall(*_, stacks)
top_crates(stacks)


'BNTZFPMMW'

In [ ]:
#| test
#overlaps = [overlap(x) for x in get_assignments(data)]
#f"{sum(overlaps):,} assignments overlap at all."

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()